In [313]:
query = """
        SELECT
          to_json_string ( STRUCT ( CAST(RANK() OVER(ORDER BY GENERATE_UUID()) AS STRING) AS id,
              STRUCT ( prdt_cd,
                store_cd ) AS
            VALUES
              )) AS
        VALUES
        FROM
          smart-ruler-304409.product_attribute_module.app_review_keywords
  """
path = "app_review_keywords"

In [314]:
import os
from pydatafabric.vault_utils import get_secrets
aws_info = get_secrets(mount_point="datafabric",path="mlops/access/data_loader")

In [315]:
bucket = "mlops-loader-dev"
destination = f"s3a://{bucket}/dynamodb/{path}/op=put/"

In [316]:
from pydatafabric.ye import get_spark

spark = get_spark()
spark.conf.set("fs.s3a.fast.upload.buffer", "disk")
spark.conf.set("fs.s3a.buffer.dir", "/tmp")
spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])

### gzip 압축 옵션 추가하여 저장

In [317]:
from pydatafabric.gcp import bq_to_df
import pyspark.sql.functions as f
import json

df = bq_to_df(query, spark_session=spark)

In [318]:
df.show()

+--------------------+
|              VALUES|
+--------------------+
|{"id":"17987","VA...|
|{"id":"43324","VA...|
|{"id":"60852","VA...|
|{"id":"67740","VA...|
|{"id":"69735","VA...|
|{"id":"72155","VA...|
|{"id":"77668","VA...|
|{"id":"80600","VA...|
|{"id":"109053","V...|
|{"id":"124978","V...|
|{"id":"129954","V...|
|{"id":"137222","V...|
|{"id":"154917","V...|
|{"id":"163171","V...|
|{"id":"167349","V...|
|{"id":"174504","V...|
|{"id":"191085","V...|
|{"id":"198363","V...|
|{"id":"232111","V...|
|{"id":"10434","VA...|
+--------------------+
only showing top 20 rows



In [319]:
from pydatafabric.gcp import bq_insert_overwrite

bq_insert_overwrite(sql=query, destination="emart-datafabric.temp_1d.app_review_keywords_json")

destination: emart-datafabric.temp_1d.app_review_keywords_json
total_rows: 237978
slot_secs: 29.831



In [320]:
from pydatafabric.gcp import import_bigquery_ipython_magic
import_bigquery_ipython_magic()

In [321]:
%%bq

SELECT *
FROM temp_1d.app_review_keywords_json

Downloading: 100%|██████████| 237978/237978 [00:00<00:00, 268186.14rows/s]

BigQuery execution took 3 seconds.


,VALUES
0,"{""id"":""6016"",""VALUES"":{""prdt_cd"":""880149237415..."
1,"{""id"":""6650"",""VALUES"":{""prdt_cd"":""111039000000..."
2,"{""id"":""16153"",""VALUES"":{""prdt_cd"":""25000001703..."
3,"{""id"":""17071"",""VALUES"":{""prdt_cd"":""11131500000..."
4,"{""id"":""21917"",""VALUES"":{""prdt_cd"":""24209600000..."
...,...
237973,"{""id"":""188093"",""VALUES"":{""prdt_cd"":""2402840000..."
237974,"{""id"":""203022"",""VALUES"":{""prdt_cd"":""8809558842..."
237975,"{""id"":""204271"",""VALUES"":{""prdt_cd"":""2440390000..."
237976,"{""id"":""209584"",""VALUES"":{""prdt_cd"":""2429230000..."


In [322]:
df = bq_to_df("SELECT * FROM temp_1d.app_review_keywords_json")
df.show()

+--------------------+
|              VALUES|
+--------------------+
|{"id":"152303","V...|
|{"id":"43539","VA...|
|{"id":"221486","V...|
|{"id":"40574","VA...|
|{"id":"35494","VA...|
|{"id":"236474","V...|
|{"id":"64984","VA...|
|{"id":"185554","V...|
|{"id":"87789","VA...|
|{"id":"101018","V...|
|{"id":"80264","VA...|
|{"id":"177357","V...|
|{"id":"70449","VA...|
|{"id":"87397","VA...|
|{"id":"19011","VA...|
|{"id":"107335","V...|
|{"id":"201156","V...|
|{"id":"191157","V...|
|{"id":"189143","V...|
|{"id":"100083","V...|
+--------------------+
only showing top 20 rows



In [326]:
df.write.mode("overwrite").option("compression", "gzip").text(destination)

22/05/03 20:32:41 WARN org.apache.hadoop.fs.s3a.commit.AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
